## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [31]:
import os
google_api_key = os.getenv('GOOGLE_API_KEY')

if google_api_key:
    print(f"GOOGLE_API_KEY Key exists and begins {google_api_key[:8]}")
else:
    print("GOOGLE_API_KEY not set - please head to the troubleshooting guide in the setup folder")
    


GOOGLE_API_KEY Key exists and begins AIzaSyCq


In [32]:
GEMINI_BASE_URL="https://generativelanguage.googleapis.com/v1beta/openai/"

In [2]:
OLLAMA_BASE_URL="http://localhost:11434/v1"


In [3]:
ollama=OpenAI(base_url=OLLAMA_BASE_URL,api_key="anything")

In [4]:
load_dotenv(override=True)
gemini=OpenAI(base_url=GEMINI_BASE_URL,api_key=google_api_key)

NameError: name 'GEMINI_BASE_URL' is not defined

In [5]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [6]:
print(linkedin)

Ankit Kolhe
Nagpur, Maharashtra· nkolhe149@gmail.com · 7666846804 · linkedin/ankit-kolhe
Education
Shri Ramdeobaba College of Engineering and Management Nagpur
B.Tech. Computer Science(Artiﬁcial Intelligence And Machine Learning)GPA: 9.51 Aug 2023 - July 2027
Shri Shivaji Science College Nagpur
HSC GPA: 81% Aug 2020 - July 2022
Bharti Krishna Vidya Vihar Nagpur
SSC GPA: 98.6% April 2010 - May 2020
Skills
Programming Languages: C++, Python, JavaScript
Framework and Libraries: React.js, Node.js, Django
Databases: MS SQL Server, MongoDB, PostgreSQL
DevOps & Tools: Git/Github, Docker
Machine Learning: TensorFlow, Pytorch, Natural Language Processing, Fine-tuning LLM, RAG
Projects
Sumy-Sumarise Python, NLP, Streamlit, scikit-learn https://sumarise-nlp.streamlit.app/
Built an NLP-based news summarization system using Latent Semantic Analysis (LSA) and sentiment
analysis, reducing article length according to user's convenience while retaining >85% key information.
Designed a complete pipeline

In [7]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [8]:
name = "Ankit Kolhe"

In [9]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [10]:
system_prompt

"You are acting as Ankit Kolhe. You are answering questions on Ankit Kolhe's website, particularly questions related to Ankit Kolhe's career, background, skills and experience. Your responsibility is to represent Ankit Kolhe for interactions on the website as faithfully as possible. You are given a summary of Ankit Kolhe's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Ankit Kolhe. I'm an diligent student and an aspiring software engineer and data scientist. I'm originally from Nagpur, India.\nI  am an INTJ, and I like all kinds of things except noisy stuff and stuff that makes no sense. \nI have loads of sarcasm and humor in store for awkward situations as well.\n\n## LinkedIn Profile:\nAnkit Kolhe\nNagpur, Maharashtra· nkolhe149@gmail.com · 7666846804 · linkedin/ankit-kolhe\nEducation

In [11]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = ollama.chat.completions.create(model="llama3.2", messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [12]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [13]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [14]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [15]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [45]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [21]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = ollama.beta.chat.completions.parse(model="llama3.2", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [22]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = ollama.chat.completions.create(model="llama3.2", messages=messages)
reply = response.choices[0].message.content

In [23]:
reply

"The patent question! Well, I'm afraid I don't hold any patents... yet! As an aspiring software engineer and data scientist, my main focus has been on learning and developing my skills rather than pursuing patent applications.\n\nHowever, I do have some pending projects and solutions that I believe could be patented in the future. My work on Sumy-Sumarise, for example, has shown promise in reducing article length while retaining key information using Latent Semantic Analysis (LSA) and sentiment analysis. If my research is further validated or improved upon, it's possible that a patent application could be pursued.\n\nBut for now, I'm focusing on mastering my craft, developing innovative solutions, and making a positive impact through technology. Who knows what the future holds? Maybe one day I'll be working with a team of experts to develop a revolutionary new product or system that earns me a spot in the patent book!"

In [24]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The response is mostly clear and concise, but could benefit from a slightly more formal tone. The tone is generally engaging and conversational, which is suitable for the context of chatting with a potential client or future employer on Ankit Kolhe's website. However, phrases like 'yet!' and 'Who knows what the future holds?' feel slightly informal for a professional context. Consider replacing them with more formal equivalents to maintain consistency.")

In [25]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = ollama.chat.completions.create(model="llama3.2", messages=messages)
    return response.choices[0].message.content

In [26]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = ollama.chat.completions.create(model="llama3.2", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Passed evaluation - returning reply
Passed evaluation - returning reply
Passed evaluation - returning reply
